# AKS Automatic + MCP Weather Server Deployment Walkthrough

This notebook orchestrates deployment of the AKS Automatic infrastructure (defined in [`main.bicep`](main.bicep)) and publishes the MCP Weather Server (located in [`mcp-weather-server/src/index.ts`](mcp-weather-server/src/index.ts)) to the cluster.

## Prerequisites
- Azure CLI authenticated: `az login`
- Docker daemon running, logged into your Azure Container Registry (ACR)
- Bicep CLI installed (comes with Azure CLI 2.50+)
- Kubernetes CLI (`kubectl`) installed
- Notebook kernel with network and shell access (e.g., Azure ML compute, local dev machine)

## 0️⃣ Initialize Notebook Variables
Before running the workflow, review these defaults and adjust them to match your Azure subscription, naming conventions, and endpoint URLs. Values fall back to environment variables when present.

In [ ]:
import os
import pathlib
from dataclasses import asdict, dataclass
from typing import Optional

@dataclass
class NotebookConfig:
    resource_group: str = "rg-aks-automatic"
    location: str = "eastus"
    aks_deployment_name: str = "aks-automatic"
    acr_name: str = "<your-registry>"
    mcp_backend_url: str = "https://<ingress-or-load-balancer-for-mcp>"
    apim_deployment_name: str = "apim-mcp"
    azure_ai_project_endpoint: str = "https://<region>.api.azureml.ms"
    azure_ai_project_id: str = "<project-guid>"
    azure_ai_resource_id: str = "/subscriptions/<sub-id>/resourceGroups/<rg>/providers/Microsoft.MachineLearningServices/workspaces/<workspace>"
    azure_openai_deployment: str = "gpt-4_1"
    apim_weather_api_url: str = "https://<service>.azure-api.net/mcp-weather"
    apim_subscription_key: str = "<optional-apim-key>"
    aks_cluster_name_override: Optional[str] = None
    apim_service_name_override: Optional[str] = None

    @classmethod
    def load(cls) -> "NotebookConfig":
        def value(name: str, default: Optional[str] = None) -> Optional[str]:
            raw = os.getenv(name)
            if raw is None or raw == "":
                return default
            return raw

        return cls(
            resource_group=value("RESOURCE_GROUP", cls.resource_group),
            location=value("LOCATION", cls.location),
            aks_deployment_name=value("AKS_DEPLOYMENT_NAME", cls.aks_deployment_name),
            acr_name=value("ACR_NAME", cls.acr_name),
            mcp_backend_url=value("MCP_BACKEND_URL", cls.mcp_backend_url),
            apim_deployment_name=value("APIM_DEPLOYMENT_NAME", cls.apim_deployment_name),
            azure_ai_project_endpoint=value("AZURE_AI_PROJECT_ENDPOINT", cls.azure_ai_project_endpoint),
            azure_ai_project_id=value("AZURE_AI_PROJECT_ID", cls.azure_ai_project_id),
            azure_ai_resource_id=value("AZURE_AI_RESOURCE_ID", cls.azure_ai_resource_id),
            azure_openai_deployment=value("AZURE_OPENAI_DEPLOYMENT", cls.azure_openai_deployment),
            apim_weather_api_url=value("APIM_WEATHER_API_URL", cls.apim_weather_api_url),
            apim_subscription_key=value("APIM_SUBSCRIPTION_KEY", cls.apim_subscription_key),
            aks_cluster_name_override=value("AKS_CLUSTER_NAME_OVERRIDE"),
            apim_service_name_override=value("APIM_SERVICE_NAME_OVERRIDE"),
        )


config = NotebookConfig.load()
CONFIG = asdict(config)

PROJECT_ROOT = pathlib.Path.cwd()
AGENT_DIR = PROJECT_ROOT / "agents"
AGENT_REQUIREMENTS = AGENT_DIR / "requirements.txt"
AGENT_SCRIPT = AGENT_DIR / "create_agent.py"
AGENT_RUN_OUTPUT = AGENT_DIR / "agent_invoke_response.json"
TEMPLATE_FILE = "main.bicep"
PARAM_FILE = "main.bicepparam"
APIM_TEMPLATE_FILE = "apim-mcp.bicep"
APIM_PARAM_FILE = "apim-mcp.bicepparam"
AGENT_API_VERSION = "2024-10-01-preview"
SUMMARY = {}

RESOURCE_GROUP = CONFIG["resource_group"]
LOCATION = CONFIG["location"]
DEPLOYMENT_NAME = CONFIG["aks_deployment_name"]
ACR_NAME = CONFIG["acr_name"]
MCP_BACKEND_URL = CONFIG["mcp_backend_url"]
APIM_DEPLOYMENT_NAME = CONFIG["apim_deployment_name"]
AZURE_AI_PROJECT_ENDPOINT = CONFIG["azure_ai_project_endpoint"]
AZURE_AI_PROJECT_ID = CONFIG["azure_ai_project_id"]
AZURE_AI_RESOURCE_ID = CONFIG["azure_ai_resource_id"]
AZURE_OPENAI_DEPLOYMENT = CONFIG["azure_openai_deployment"]
APIM_WEATHER_API_URL = CONFIG["apim_weather_api_url"]
APIM_SUBSCRIPTION_KEY = CONFIG["apim_subscription_key"]
AKS_CLUSTER_NAME_OVERRIDE = CONFIG.get("aks_cluster_name_override") or None
APIM_SERVICE_NAME_OVERRIDE = CONFIG.get("apim_service_name_override") or None
IMAGE_NAME = f"{ACR_NAME}.azurecr.io/mcp-weather-server:latest"

print("Notebook configuration:\n")
for key, value in CONFIG.items():
    display = value if value not in (None, "") else "[not set]"
    if key.endswith("_key") and display not in ("[not set]", None, "") and "<" not in str(display):
        display = f"****{str(display)[-4:]}"
    print(f"  {key}: {display}")

print("\nArtifacts:")
print(f"  project_root: {PROJECT_ROOT}")
print(f"  agents_dir: {AGENT_DIR}")
print(f"  template_file: {TEMPLATE_FILE}")
print(f"  container_image: {IMAGE_NAME}")

print("\n✅ Notebook initialized.")

In [ ]:
import json
import os
import pathlib
import subprocess
import tempfile
import time
from typing import List, Optional, Sequence

import requests
from azure.identity import DefaultAzureCredential


def get_access_token(scope: str = "https://cognitiveservices.azure.com/.default") -> str:
    """Acquire an Azure AD token for the specified scope."""
    credential = DefaultAzureCredential()
    token = credential.get_token(scope)
    return token.token


def run_command(command: Sequence[str], *, cwd: Optional[str] = None, capture_output: bool = False, text: bool = True, env: Optional[dict] = None) -> subprocess.CompletedProcess:
    """Run a CLI command with consistent logging."""
    printable = " ".join(command)
    print(f"$ {printable}")
    return subprocess.run(command, cwd=cwd, env=env, check=True, capture_output=capture_output, text=text)


def deploy_group_bicep(deployment_name: str, resource_group: str, template_file: str, parameter_file: str, extra_parameters: Optional[List[str]] = None) -> dict:
    """Deploy a Bicep template to a resource group and return the deployment outputs."""
    args = [
        "az", "deployment", "group", "create",
        "--name", deployment_name,
        "--resource-group", resource_group,
        "--template-file", template_file,
        "--parameters", f"@{parameter_file}",
        "--output", "json",
    ]
    if extra_parameters:
        args.extend(extra_parameters)
    completed = run_command(args, capture_output=True)
    return json.loads(completed.stdout)

print("Loaded core libraries and helper utilities.")

In [ ]:
# Step 1: Ensure the resource group exists

run_command([
    "az", "group", "create",
    "--name", RESOURCE_GROUP,
    "--location", LOCATION,
    "--output", "none",
])

print("✔️ Resource group ready.")

In [ ]:
# Step 2: Deploy AKS Automatic using Bicep

deployment_result = deploy_group_bicep(
    DEPLOYMENT_NAME,
    RESOURCE_GROUP,
    TEMPLATE_FILE,
    PARAM_FILE,
 )

deployment_outputs = deployment_result.get("properties", {}).get("outputs", {})

print(json.dumps(deployment_outputs, indent=2))

print("✔️ AKS deployment completed.")

In [ ]:
# Step 3: Capture key outputs for later steps

cluster_name = AKS_CLUSTER_NAME_OVERRIDE or deployment_outputs.get("clusterName", {}).get("value")
workspace_id = deployment_outputs.get("workspaceId", {}).get("value")
cluster_id = deployment_outputs.get("clusterId", {}).get("value") if deployment_outputs else None
print(f"Cluster name: {cluster_name}")
print(f"Log Analytics workspace: {workspace_id}")
if not cluster_name:
    raise ValueError("clusterName output missing; check your deployment parameters and outputs.")
SUMMARY.update({
    "aks_cluster_name": cluster_name,
    "aks_cluster_id": cluster_id,
    "log_analytics_workspace": workspace_id,
})
print("✔️ Captured AKS outputs.")

In [ ]:
# Step 4: Attach ACR permissions to the AKS cluster

run_command([
    "az", "aks", "update",
    "--resource-group", RESOURCE_GROUP,
    "--name", cluster_name,
    "--attach-acr", ACR_NAME,
])

print("✔️ AKS cluster now has pull permissions for the ACR.")

In [ ]:
# Step 5: Merge AKS credentials into local kubeconfig

run_command([
    "az", "aks", "get-credentials",
    "--resource-group", RESOURCE_GROUP,
    "--name", cluster_name,
    "--overwrite-existing",
])

print(f"✔️ Kubeconfig updated for {cluster_name}.")

In [ ]:
# Step 6: Build and push MCP Weather Server image

server_dir = PROJECT_ROOT / "mcp-weather-server"
run_command(["npm", "install"], cwd=str(server_dir))
run_command(["npm", "run", "build"], cwd=str(server_dir))
run_command(["docker", "build", "-t", IMAGE_NAME, "."], cwd=str(server_dir))
run_command(["az", "acr", "login", "--name", ACR_NAME])
run_command(["docker", "push", IMAGE_NAME])
print("✔️ Container image built and pushed.")

In [ ]:
# Step 7: Deploy MCP Weather Server workload to AKS

manifest_path = PROJECT_ROOT / "k8s" / "mcp-weather.yaml"
manifest_text = manifest_path.read_text()
patched_manifest = manifest_text.replace("<your-registry>.azurecr.io/mcp-weather-server:latest", IMAGE_NAME)
with tempfile.NamedTemporaryFile("w", suffix=".yaml", delete=False) as temp_manifest:
    temp_manifest.write(patched_manifest)
    rendered_manifest_path = temp_manifest.name
run_command(["kubectl", "apply", "-f", rendered_manifest_path])
print("✔️ MCP Weather workload applied to AKS.")

In [ ]:
# Step 8: Verify rollout

run_command(["kubectl", "get", "pods", "-l", "app=mcp-weather-server"])
try:
    run_command(["kubectl", "logs", "deploy/mcp-weather-server"])
except subprocess.CalledProcessError as exc:
    print(f"kubectl logs failed: {exc}. Continuing.")
print("✔️ Reviewed pod status and logs (see output above).")

## Publish MCP Weather through API Management
Set the MCP service endpoint (`MCP_BACKEND_URL`) above to an address reachable by API Management (for example, an Azure Application Gateway, load balancer IP, or ingress hostname that fronts the MCP Weather service). Then run the following steps to provision API Management and wire it to the backend.

In [ ]:
# Step 9: Deploy API Management for MCP Weather

if "<ingress" in MCP_BACKEND_URL or MCP_BACKEND_URL.endswith("for-mcp>"):
    raise ValueError("Set MCP_BACKEND_URL to the live endpoint that fronts the MCP Weather service before deploying APIM.")

apim_deploy = deploy_group_bicep(
    APIM_DEPLOYMENT_NAME,
    RESOURCE_GROUP,
    APIM_TEMPLATE_FILE,
    APIM_PARAM_FILE,
    extra_parameters=["--parameters", f"mcpBackendUrl={MCP_BACKEND_URL}"],
)

apim_outputs = apim_deploy.get("properties", {}).get("outputs", {})

print(json.dumps(apim_outputs, indent=2))

print("✔️ API Management deployment completed.")

In [ ]:
# Step 10: Summarize API Management endpoints

apim_parameters = apim_deploy.get("properties", {}).get("parameters", {})
apim_service_name = APIM_SERVICE_NAME_OVERRIDE or apim_parameters.get("serviceName", {}).get("value")
print(f"APIM service name: {apim_service_name}")
print("Gateway URL:", apim_outputs.get("gatewayUrl", {}).get("value"))
print("Developer portal URL:", apim_outputs.get("developerPortalUrl", {}).get("value"))
print("Weather API invoke URL:", apim_outputs.get("apiInvokeUrl", {}).get("value"))
print("✔️ Recorded API Management endpoints.")

In [ ]:
# Step 11: Store APIM outputs for later steps

if not apim_outputs:
    raise ValueError("APIM outputs unavailable. Run Step 9 to deploy API Management first.")

apim_gateway_url = apim_outputs.get("gatewayUrl", {}).get("value")
apim_developer_portal = apim_outputs.get("developerPortalUrl", {}).get("value")
apim_weather_api_url = apim_outputs.get("apiInvokeUrl", {}).get("value")

if apim_weather_api_url:
    APIM_WEATHER_API_URL = apim_weather_api_url
    os.environ["APIM_WEATHER_API_URL"] = APIM_WEATHER_API_URL
    SUMMARY["apim_weather_api_url"] = APIM_WEATHER_API_URL
    print(f"Stored APIM weather API URL: {APIM_WEATHER_API_URL}")
else:
    print("APIM weather API URL not found in outputs; ensure apim-mcp.bicep succeeded.")

if apim_gateway_url:
    SUMMARY["apim_gateway_url"] = apim_gateway_url
    print(f"APIM gateway URL: {apim_gateway_url}")
if apim_developer_portal:
    SUMMARY["apim_developer_portal"] = apim_developer_portal
    print(f"APIM developer portal: {apim_developer_portal}")
print("✔️ APIM outputs cached for downstream steps.")

## Create Azure AI Foundry Agent
Provide Azure AI Foundry project details in the configuration cell (Cell 3) or export them as environment variables before running the steps below. The agent script will call the Azure AI Projects Agents API to register a GPT-4.1 agent that invokes the MCP Weather API via API Management.

In [ ]:
# Step 12: Install agent dependencies

if not AGENT_REQUIREMENTS.exists():
    raise FileNotFoundError(f"Missing requirements file at {AGENT_REQUIREMENTS}.")

run_command(["python", "-m", "pip", "install", "-r", str(AGENT_REQUIREMENTS)])

print("✔️ Agent dependencies installed.")

In [ ]:
# Step 13: Create GPT-4.1 agent in Azure AI Foundry

placeholders = [AZURE_AI_PROJECT_ENDPOINT, AZURE_AI_PROJECT_ID, AZURE_AI_RESOURCE_ID, AZURE_OPENAI_DEPLOYMENT, APIM_WEATHER_API_URL]
if any(token.startswith("http") and "<" in token for token in placeholders) or any("<" in token for token in placeholders):
    raise ValueError("Replace Azure AI / APIM configuration placeholders in the configuration cell or set real values via environment variables before running this step.")

if not AGENT_SCRIPT.exists():
    raise FileNotFoundError(f"Agent script missing at {AGENT_SCRIPT}.")

agent_env = os.environ.copy()
agent_env.update({
    "AZURE_AI_PROJECT_ENDPOINT": AZURE_AI_PROJECT_ENDPOINT,
    "AZURE_AI_PROJECT_ID": AZURE_AI_PROJECT_ID,
    "AZURE_AI_RESOURCE_ID": AZURE_AI_RESOURCE_ID,
    "AZURE_OPENAI_DEPLOYMENT": AZURE_OPENAI_DEPLOYMENT,
    "APIM_WEATHER_API_URL": APIM_WEATHER_API_URL,
})
if APIM_SUBSCRIPTION_KEY and "<" not in APIM_SUBSCRIPTION_KEY:
    agent_env["APIM_SUBSCRIPTION_KEY"] = APIM_SUBSCRIPTION_KEY

run_command(["python", AGENT_SCRIPT.name], cwd=str(AGENT_DIR), env=agent_env)

agent_response_file = AGENT_DIR / "agent_response.json"
if agent_response_file.exists():
    agent_payload = json.loads(agent_response_file.read_text())
    agent_identifier = agent_payload.get("id") or agent_payload.get("name") or agent_payload.get("agentId")
    SUMMARY["agent_id"] = agent_identifier
    print(f"Agent ID: {agent_identifier}")
else:
    print("agent_response.json not found; ensure the helper script completed successfully.")

print("✔️ Agent creation request submitted. Inspect agents/agent_response.json for additional metadata.")

## Test the Azure AI Foundry Agent
Use the agent identifier returned in `agents/agent_response.json` to send a test weather query.

In [ ]:
# Step 14: Invoke the agent with a sample weather request

agent_response_path = AGENT_DIR / "agent_response.json"
if not agent_response_path.exists():
    raise FileNotFoundError("Run Step 13 first so agent_response.json is available.")

agent_data = json.loads(agent_response_path.read_text())
agent_id = agent_data.get("id") or agent_data.get("name") or agent_data.get("agentId")
if not agent_id:
    raise ValueError("Agent identifier not found in agent_response.json. Inspect the file and update this cell if necessary.")

for value in [AZURE_AI_PROJECT_ENDPOINT, AZURE_AI_PROJECT_ID, AZURE_AI_RESOURCE_ID]:
    if value is None or "<" in value:
        raise ValueError("Provide valid Azure AI configuration values (configuration cell or environment variables) before invoking the agent.")

token = get_access_token()
headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json",
    "ai-resource-id": AZURE_AI_RESOURCE_ID,
    "ai-project-id": AZURE_AI_PROJECT_ID,
}

invoke_url = f"{AZURE_AI_PROJECT_ENDPOINT}/openai/agents/{agent_id}:invoke?api-version={AGENT_API_VERSION}"
payload = {
    "input": [
        {
            "role": "user",
            "content": "What is the weather like today in Seattle?"
        }
    ]
}

response = requests.post(invoke_url, headers=headers, data=json.dumps(payload), timeout=60)
response.raise_for_status()
invoke_result = response.json()
AGENT_RUN_OUTPUT.write_text(json.dumps(invoke_result, indent=2))

assistant_reply = None
output = invoke_result.get("output") or invoke_result.get("result")
if isinstance(output, list):
    for item in output:
        blocks = item.get("content") if isinstance(item, dict) else None
        if isinstance(blocks, list):
            for block in blocks:
                if isinstance(block, dict):
                    if block.get("type") == "text":
                        assistant_reply = block.get("text") or block.get("value")
                    elif "text" in block and isinstance(block["text"], dict):
                        assistant_reply = block["text"].get("value")
                if assistant_reply:
                    break
        elif isinstance(blocks, str):
            assistant_reply = blocks
        if assistant_reply:
            break
elif isinstance(output, dict):
    assistant_reply = output.get("message") or output.get("text")

SUMMARY["agent_invoke_response_file"] = str(AGENT_RUN_OUTPUT)
if assistant_reply:
    SUMMARY["agent_reply"] = assistant_reply

print(f"Agent invocation response saved to {AGENT_RUN_OUTPUT}")
if assistant_reply:
    print("\nAgent reply:\n")
    print(assistant_reply)
else:
    print(json.dumps(invoke_result, indent=2))

print("✔️ Agent invocation completed.")

## Deployment Summary
Collected outputs from prior steps for quick reference.

In [ ]:
# Display collected outputs for reference
for key, value in SUMMARY.items():
    if value:
        print(f"{key}: {value}")
    else:
        print(f"{key}: [not available]")

## Next steps
- Attach your MCP-compatible Agent to the running pod (sidecar or exec).
- Harden networking, RBAC, and secret management for production environments.
- Configure API Management products, subscriptions, and policies to suit your consumers.
- Extend the notebook with teardown steps (`kubectl delete`, `az group delete`) when you're ready to clean up.